<a href="https://colab.research.google.com/github/anacgfp/Computacao-Natural/blob/master/ga_desafio_CN_2020_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Você deverá construir um modelo, utilizando Algoritmo Genético, para resolver a demanda de entregas das Lojas do Recife. Porém, este seu modelo deve seguir a lógica do Problema do Caixeiro Viajante (PCV), onde é preciso determinar a menor rota para percorrer uma série de cidades (visitando uma única vez cada uma delas), e retornar à cidade de origem.

**Considerações**



1.   A função de aptidão (fitness) será 1 / d, onde “d” é a distância total percorrida.
2.   A distância pode ser calculada usando a distância euclidiana entre as cidades.
3.   A sua solução deve sempre gerar rotas sintaticamente corretas, isto é, as rotas devem visitar cada cidade apenas uma vez, retornando à cidade de origem.



**Entrega**

Você deverá entregar um relatório contendo:
1. Comparação entre os resultados obtidos pelos diferentes métodos de seleção (roleta, torneio e elitismo);
2. Validar ao menos dois operadores de crossover e comentar seus resultados;
3. Validar ao menos dois operadores de mutação e comentar seus resultados;
4. Exibir a Rota Final


In [ ]:
import pandas as pd
import numpy as np
import math
import matplotlib as mplib
import random
import copy 

In [ ]:
#parameters and constants

pop_size = 10
pm = 1
generations = 10
selection_op = 'elitism'
crossover_op = 'single point'
mutation_op = 'random'

In [ ]:
#Problem representation 
bairros_list = ["Aflitos",
"Casa Amarela",
"Casa Forte",
"Campo Grande",
"Boa Viagem",
"Piedade",
"Cidade Universitária",
"Derby",
"Encruzilhada",
"Ibura"]
x_list = [50,140,110,30,240,400,330,80,30,280]
y_list = [60,78,55,108,378,128,256,15,20,12]

In [ ]:
class Bairro:
  def __init__(self, nome, x, y):
    self.nome = nome
    self.x = x
    self.y = y

#euclidian distance between two neighborhoods
  def get_dist(self, bairro):
    return math.sqrt((bairro.x - self.x)**2 + (bairro.y - self.y)**2)
  
  def print_bairro(self):
    print(self.nome, ' ', self.x, ' ', self.y)

def inicializar_bairros():
  bairros = []
  for i in range(0, len(bairros_list)):
    bairros.append(Bairro(bairros_list[i], x_list[i], y_list[i]))
  return bairros

BAIRROS = inicializar_bairros()

In [ ]:
class Individual:
  def __init__(self, bairros):
    self.bairros = bairros
    self.fitness = self.get_fitness()

  #Calculating the fitness of the the individual (route)
  def get_fitness(self):
    distance = 0
    for i in range(0, len(self.bairros)):
      if(i == len(self.bairros) -1):
        distance += self.bairros[i].get_dist(self.bairros[0])
      else:
        distance += self.bairros[i].get_dist(self.bairros[i+1])
    return 1/distance

  #printing individuals
  def print_ind(self):
    for i in self.bairros:
     i.print_bairro()
    
  #plotting individuals
  def plot_ind(self):
    aux_x = []
    aux_y = []
    for i in self.bairros:
      aux_x.append(i.x)
      aux_y.append(i.y)
    aux_x.append(self.bairros[0].x)
    aux_y.append(self.bairros[0].y)
    mplib.pyplot.scatter(aux_x, aux_y) 
    mplib.pyplot.plot(aux_x, aux_y) 

In [ ]:
#generate the population
def init_pop():
  pop = []
  for _ in range(0, pop_size):
    bairros = copy.deepcopy(BAIRROS)
    np.random.shuffle(bairros)    
    pop.append(Individual(bairros))
  return pop

class Population:
  def __init__(self, pop = init_pop()):
    self.pop_size = pop_size
    self.pop = pop

  def get_best(self):
    best_individual = self.pop[0]
    for i in self.pop:
      if i.get_fitness() > best_individual.get_fitness():
        best_individual = i
    return best_individual

  def print_pop(self):
    for i in self.pop:
      i.print_ind()
      print('------------------------------------------')

In [ ]:
def get_fitness_list(pop):
  fitness_list = []
  for i in range(0, len(pop)):
    fitness_list.append(pop[i].get_fitness())
  return fitness_list


def get_prob_distr(pop, fitness_list):
  prob_distr = []
  a = 0
  total_fitness = sum(fitness_list)
  for i in fitness_list:
    a += i/total_fitness
    prob_distr.append(a)
  return prob_distr

def roulette(population):
  prob_distr = get_prob_distr(population, fitness_list = get_fitness_list(population) )
  parents = []
  n = 0
  while n < 2:
    r = random.random()
    i = 0
    while  prob_distr[i] < r:
      i += 1
    parents.append( population[i])
    n += 1
  return [parents[0], parents[1]]

In [ ]:
def tournament(pop):
  k = 10
  current_member = 1
  mating_pool = []
  while current_member <= 2:
    new_pop = Population(k, random.sample(pop, k))
    best = new_pop.get_best()
    mating_pool.append(best)
    current_member += 1
  return [mating_pool[0], mating_pool[1]]

def elitism(population):
  mating_pool = population.pop.copy()
  mating_pool.remove(population.get_best())
  return [mating_pool[0], mating_pool[1]]

selection_operator = {
    'roulette': roulette,
    'tournament': tournament,
    'elitism': elitism
}

In [ ]:
def ox_crossover(parent1, parent2):
  parent1 = parent1.bairros
  parent2 = parent2.bairros
  #calculate initial index of substring and the size of substring randomly
  index = random.randint(0, len(parent1)-1)
  size = random.randint(1, 4)
  #calculate the random substring
  subst = parent1[index: (index+size)]
  #create the proto child
  proto_child = []
  i = 0
  while len(proto_child) < index:
    if not contains(subst, parent2[i]):
      proto_child.append(parent2[i])
    i += 1      
  for k in subst : 
    proto_child.append(k)
  while len(proto_child) < len(parent2):
    if not contains(proto_child, parent2[i]):
      proto_child.append(parent2[i])
    i += 1
  print('2', i, parent2[i].nome)
  offspring = Individual(proto_child)
  return offspring

def contains(lista_de_bairros, b):
  for i in lista_de_bairros:
    if i.nome == b.nome:
      return True
  return False



[9, 10]


In [ ]:
def ga(selection, crossover, mutation):
  population = Population()
  best = population.get_best()
  g = 0
  while g < generations:
    aux_pop = []
    while len(aux_pop) < 1:
      [parent1, parent2] = selection_operator[selection](population.pop)
      offspring = crossover_operator[crossover](parent1, parent2)
      # print('---------------parent1---------------')
      # parent1.print_ind()
      # print('--------------------------------------')
      # print('---------------parent2---------------')
      # parent2.print_ind()
      # print('--------------------------------------')
      # print('---------------individuo---------------')
      # offspring.print_ind()
      # print('--------------------------------------')
      n = random.random()
      if n < pm:
        offspring = mutation_operator[mutation](offspring)
      aux_pop.append(offspring)
    population = Population(aux_pop)
    if population.get_best().get_fitness() > best.get_fitness():
      best = population.get_best()    
    g += 1
  return best

resultado = ga('roulette', 'ox','insertion')
print(1/resultado.get_fitness())
resultado.plot_ind()


Cidade Universitária
Boa Viagem
Casa Forte
Ibura
Derby
Encruzilhada
Piedade
Casa Amarela
Campo Grande
Casa Forte
Aflitos
Cidade Universitária
Boa Viagem
Ibura
Derby
Casa Amarela
Encruzilhada
Piedade
Casa Forte
Aflitos
Cidade Universitária
Ibura
Derby
Casa Amarela
Encruzilhada
Piedade
Campo Grande
Casa Forte
Encruzilhada
Aflitos
Cidade Universitária
Ibura
Derby
Casa Amarela
Piedade
Campo Grande
Casa Forte
Encruzilhada
Cidade Universitária
Ibura
Derby
Casa Amarela
Piedade
Campo Grande
Aflitos
Casa Forte
Encruzilhada
Cidade Universitária
Ibura
Derby
Casa Amarela
Piedade
Aflitos
Aflitos


IndexError: ignored